# RAGの仕組み説明プログラム

このノートブックでは、RAG（Retrieval-Augmented Generation）の仕組みを説明するためのプログラムを実装します。
Wordファイルからドキュメントを読み込み、テキスト検索、ベクトル検索、ハイブリッド検索、セマンティックランキングの結果を比較し、
OpenAI APIと連携してRAGの効果を示します。

In [0]:
%pip install docx2txt
%pip install --upgrade langchain-community
%pip install --upgrade langchain
%pip install --upgrade sqlalchemy
%pip install sqlalchemy==1.4.46
%pip install --upgrade pgvector
%pip install faiss-cpu
%pip install rank_bm25

In [0]:
# 必要なライブラリのインポート
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import Docx2txtLoader
from langchain.retrievers import BM25Retriever
from langchain.retrievers import MergerRetriever
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage


## ベクトルストア作成

RAGが検索するWordドキュメントを読み込み、ベクトル化します。

In [0]:
# Wordドキュメントの読み込み
loader = Docx2txtLoader("./profeel.docx")
documents = loader.load()

# ドキュメントの分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=51)  # 10%オーバーラップ
texts = text_splitter.split_documents(documents)

# ベクトル化
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

print(f"ベクトル化されたドキュメント数: {len(texts)}")

### チャンク分割した内容を確認

In [0]:
texts[0]

In [0]:
texts[1]

In [0]:
texts[2]

## RAGによる検索

テキスト検索、ベクトル検索、ハイブリッド検索、セマンティックランキングの結果を取得・表示します。

In [0]:
# 検索クエリ
query = "小磯さんは製薬会社の時に何をしていましたか？"
query = "小磯さんは女性向けの薬について何か知ってる？"


# テキスト検索（BM25）
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_results = bm25_retriever.get_relevant_documents(query)

print("★1 テキスト検索結果:")
for i, doc in enumerate(bm25_results[:3], 1):
    print(f"{i}. {doc.page_content[:100]}...")

# ベクトル検索
vector_results = vectorstore.similarity_search(query)

print("\n★2 ベクトル検索結果:")
for i, doc in enumerate(vector_results[:3], 1):
    print(f"{i}. {doc.page_content[:100]}...")

# ハイブリッド検索
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vectorstore.as_retriever()],
    weights=[0.5, 0.5]
)
hybrid_results = hybrid_retriever.get_relevant_documents(query)

print("\n★3 ハイブリッド検索結果:")
for i, doc in enumerate(hybrid_results[:3], 1):
    print(f"{i}. {doc.page_content[:100]}...")

# セマンティックランカーによる並べ替え
llm = ChatOpenAI(temperature=0, model="gpt-4")
compressor = LLMChainExtractor.from_llm(llm)
reranked_results = compressor.compress_documents(hybrid_results, query)

print("\n★4 セマンティックランカーで並べ替えた結果:")
for i, doc in enumerate(reranked_results[:3], 1):
    print(f"{i}. {doc.page_content[:100]}...")

## OpenAI APIとの連携

RAGにて取得した各種データを使用してプロンプトを組み立て、OpenAI APIに送信します。

In [0]:
# ChatGPTモデルの初期化
chat = ChatOpenAI(
    model_name="gpt-4",
    temperature=0
)

# プロンプトテンプレートの作成
def create_prompt(retrieval_results, query):
    return ChatPromptTemplate.from_messages([
        SystemMessage(content="あなたは親切なアシスタントです。与えられた情報を元に質問に答えてください。"),
        HumanMessage(content=f"以下の情報を参考に、質問に答えてください:\n\n" + 
                             "\n\n".join([doc.page_content for doc in retrieval_results[:3]]) +
                             f"\n\n質問: {query}")
    ])

# 各検索方法のプロンプト作成と表示、APIへの送信
search_methods = [
    ("テキスト検索", bm25_results),
    ("ベクトル検索", vector_results),
    ("ハイブリッド検索", hybrid_results),
    ("セマンティックランキング", reranked_results)
]

responses = {}

for method_name, results in search_methods:
    prompt = create_prompt(results, query)
    print(f"\n{method_name}のプロンプト:")
    print(prompt.format_messages()[1].content)
    
    # OpenAI APIに送信
    response = chat(prompt.format_messages())
    responses[method_name] = response.content

# RAGなしのプロンプト作成と送信
no_rag_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="あなたは親切なアシスタントです。質問に答えてください。"),
    HumanMessage(content=f"質問: {query}")
])

print("\nRAGなしのプロンプト:")
print(no_rag_prompt.format_messages()[1].content)

# RAGなしの応答を取得
no_rag_response = chat(no_rag_prompt.format_messages())
responses["RAGなし"] = no_rag_response.content

## 回答結果比較

RAGなしの回答結果と各RAG手法を使用した回答結果を表示します。

In [0]:
for method, response in responses.items():
    print(f"\n{method}の回答結果:")
    print(response)